<a href="https://colab.research.google.com/github/sbooeshaghi/azucar/blob/main/reference/brain/celltype/markers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install --quiet anndata

     |████████████████████████████████| 96 kB 2.8 MB/s 


In [1]:
!curl -Ls https://github.com/sbooeshaghi/azucar/blob/main/reference/brain/celltype/markers.h5ad?raw=true > markers.h5ad

In [40]:
import anndata
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.preprocessing import normalize, scale
import glob

import numpy.lib.recfunctions as rf

def nd(arr):
    return np.asarray(arr).reshape(-1)

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

In [6]:
adata = anndata.read_h5ad("markers.h5ad")

In [18]:
adata

AnnData object with n_obs × n_vars = 5690 × 39
    obs: 'barcodes', 'celltype'

In [66]:
mg_mean = pd.DataFrame(adata.X, index=adata.obs["celltype"], columns=adata.var.index.values).groupby("celltype").mean()
top_idx = np.argsort(mg_mean.values, 1)[:,-10:]

In [75]:
# positive value and top 10
d = defaultdict(list)
for idx, (ct, t) in enumerate(zip(mg_mean.index.values, top_idx)):
  mask = mg_mean.loc[ct].values[t] > 0
  d[ct] = adata.var.index.values[t][mask]

In [77]:
[len(v) for k,v in d.items()]

[6, 8, 5, 7, 9, 5, 10, 5, 5, 4, 8, 6, 6, 9, 6, 5, 5, 6, 8, 6, 7, 3, 6, 6]

In [78]:
d

defaultdict(list,
            {'Astro1': array(['Olig1', 'Gfap', 'Hepacam', 'Cxcl14', 'Apoe', 'Aldoc'],
                   dtype=object),
             'Astro2': array(['Plp1', 'Olig1', 'Reln', 'Cxcl14', 'Hepacam', 'Apoe', 'Aldoc',
                    'Gfap'], dtype=object),
             'COPs': array(['Olig1', 'C1ql1', 'Olig2', 'Neu4', 'Plp1'], dtype=object),
             'ChorPlex': array(['Hydin', 'Riiad1', 'Apoe', 'Foxj1', 'Slc4a5', 'Sostdc1', 'Aqp1'],
                   dtype=object),
             'DivOPCs': array(['Neu4', 'Cxcl14', 'C1ql1', 'Olig1', 'Olig2', 'Rrm2', 'Pdgfra',
                    'Top2a', 'Ccna2'], dtype=object),
             'Endo': array(['Top2a', 'Ccna2', 'Kcnj8', 'Ptprb', 'Cldn5'], dtype=object),
             'Ependymal': array(['Aldoc', 'Gfap', 'Reln', 'Hepacam', 'Top2a', 'Apoe', 'Npy',
                    'Hydin', 'Riiad1', 'Foxj1'], dtype=object),
             'ExcitNeuron1': array(['Npy', 'Dcn', 'Bhlhe22', 'Grin2b', 'Neurod6'], dtype=object),
             '